## 1. Loading and Exploring Time Series Data
We'll use **sktime** datasets for richer time series analysis.

In [ ]:
from sktime.datasets import load_airline
import pandas as pd
df = load_airline()
df = pd.DataFrame(df)
df.columns = ['y']
df.head()

## 2. Simulating and Handling Missing Data
We'll create missing values and explore **multiple imputation methods**.

In [ ]:
import numpy as np
np.random.seed(0)
m = df.shape[0]
mask = np.random.choice([True, False], size=m, p=[0.1,0.9])
df_missing = df.copy()
df_missing.loc[mask] = np.nan
df_missing.isna().sum()

In [ ]:
df_ffill = df_missing.fillna(method='ffill')
df_ffill.plot(title='Forward Fill Imputation')

In [ ]:
df_bfill = df_missing.fillna(method='bfill')
df_bfill.plot(title='Backward Fill Imputation')

In [ ]:
df_interp = df_missing.interpolate()
df_interp.plot(title='Interpolation Imputation')

## 3. Resampling and Aggregation
We'll aggregate monthly data to quarterly.

In [ ]:
df_q_sum = df_ffill.resample('Q').sum()
df_q_mean = df_ffill.resample('Q').mean()
df_q_sum.head(), df_q_mean.head()

## 4. Decomposition
Split data into trend, seasonality, residuals using sktime.

In [ ]:
from sktime.transformations.series.detrend import STLTransformer
stl = STLTransformer() 
df_stl = stl.fit_transform(df_ffill)
df_stl.plot(title='STL Decomposition')

## 5. Forecasting with sktime Models
Fit multiple forecasting models and compare.

In [ ]:
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.model_selection import temporal_train_test_split

y = df_ffill['y']
y_train, y_test = temporal_train_test_split(y, test_size=12)

forecaster = NaiveForecaster(strategy='last')
forecaster.fit(y_train)
y_pred = forecaster.predict(fh=list(range(1,13)))
y_pred

In [ ]:
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
forecaster2 = ExponentialSmoothing(trend='add', seasonal='add', sp=12)
forecaster2.fit(y_train)
y_pred2 = forecaster2.predict(fh=list(range(1,13)))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
y_train.plot(label='Train')
y_test.plot(label='Test')
y_pred.plot(label='Naive Forecast')
y_pred2.plot(label='Exponential Smoothing')
plt.legend();

## 6. Evaluation
Compute metrics for both models.

In [ ]:
from sktime.performance_metrics.forecasting import mean_absolute_error, mean_absolute_percentage_error
mae_naive = mean_absolute_error(y_test, y_pred)
mape_naive = mean_absolute_percentage_error(y_test, y_pred)
mae_exp = mean_absolute_error(y_test, y_pred2)
mape_exp = mean_absolute_percentage_error(y_test, y_pred2)
print('Naive MAE:', mae_naive, 'MAPE:', mape_naive)
print('Exp Smoothing MAE:', mae_exp, 'MAPE:', mape_exp)